# SPEED
## speed tests

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split, Subset
from sklearn.preprocessing import LabelEncoder
from IPython.display import clear_output
import os
import time
import random
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from models import *
from data_loader import *
from data_format import *
from datapath import *
from IPython.display import clear_output
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:18230"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device=torch.device("cpu")
print(f"Using device: {device}")
csfont = {'fontname':'Times New Roman'}
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
#


Using device: cuda
Using device: cuda
Using device: cuda


## create dataset

In [8]:
#get varying sizes of the dataset
"""data=loaded(t=-1,filename="X_data_15.npz")
data.applySobel()
data.orientation_augment()
data.speed_augment(4,speeds=3)
data.zoom_augment([10,20,30,40])"""

'data=loaded(t=-1,filename="X_data_15.npz")\ndata.applySobel()\ndata.orientation_augment()\ndata.speed_augment(4,speeds=3)\ndata.zoom_augment([10,20,30,40])'

## experiments

In [2]:
#global parameters
test_data=np.load("/its/home/drs25/RoboSkin/Code/NewRigExperiments/texture-tactip/concatenated_data_handheld.npy").astype(np.uint8)
labels=np.load("/its/home/drs25/RoboSkin/Code/NewRigExperiments/texture-tactip/concatenated_labels_handheld.npy").astype(np.uint8)
test_data_sliced=test_data[:,::4,:,:]
test_data_sliced=test_data_sliced[:,10:10+4,:,:]
crop=[60,180,40,150]
im=test_data_sliced[:,:,crop[2]:crop[3],crop[0]:crop[1]]
new=np.zeros((im.shape[0],im.shape[1],im.shape[2],im.shape[3]))
for i in range(len(test_data_sliced)): #crop all images individually
    for j in range(len(test_data_sliced[0])):
        image=cv2.resize(test_data_sliced[i][j],(int(640*0.4),int(480*0.4)),interpolation=cv2.INTER_AREA)
        image=cv2.cvtColor(image[crop[2]:crop[3],crop[0]:crop[1]], cv2.COLOR_BGR2GRAY)
        # Apply Sobel filter in x-direction
        sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)  # ksize=3 for a 3x3 Sobel kernel

        # Apply Sobel filter in y-direction
        sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)

        # Convert the results back to uint8
        sobel_x = np.uint8(np.absolute(sobel_x))
        sobel_y = np.uint8(np.absolute(sobel_y))

        # Combine the results to get the final edge-detected image
        sobel_combined = cv2.bitwise_or(sobel_x, sobel_y)
        new[i][j]=sobel_combined
print(new.shape)
new=new.reshape(new.shape[0],1,4*new.shape[2],new.shape[3])
print(new.shape)
new=(new-np.mean(new))/(np.max(new)-np.min(new))
test_data_sliced_tensor=torch.tensor(new,dtype=torch.float64).float().to(device)


def calc_noisy(preds,labels):
    names=['Carpet', 'LacedMatt', 'wool', 'Cork', 'Felt', 'LongCarpet', 'cotton', 'Plastic', 'Flat', 'Ffoam', 'Gfoam', 'bubble', 'Efoam', 'jeans', 'Leather']
    the_other_labels=['Cork','FoamF','FoamG','LacedMatt','Leather','LongCarpet','Plastic','ShortCarpet','Bubble','FoamE']
    mapping={0:7,1:3,3:0,5:5,7:6,9:1,10:2,11:8,12:9,14:4}
    classes=torch.argmax(preds,axis=1)
    correct=0
    for i in range(len(classes)):
        if mapping.get(classes[i],-1) >0:
            if mapping[classes[i]]==labels[i]: correct+=1
    return correct/len(classes) *100

num_trials=20
T_SIZE=4
EPOCHS=100

(50, 4, 110, 120)
(50, 1, 440, 120)


### experiment with different zooms

In [33]:
## zoom experiments
acc=np.zeros((num_trials,3)) #train,test,noisy

for i in range(num_trials):
    data=loaded(t=T_SIZE,filename="X_data_15.npz")
    data.applySobel()
    data.zoom_augment([10,20,30,40])
    data.shuffle()
    data.X=data.X[0:15000]
    data.y=data.y[0:15000]
    train_loader,test_loader=org_data(data,(len(data.X),1,T_SIZE*data.X.shape[2],data.X.shape[3]))
    model,history=run(train_loader,0,4,num_epochs = EPOCHS)
    print("Commence calculations...")
    acc[i][0]=calc(model,train_loader)
    acc[i][1]=calc(model,test_loader)
    preds=model(test_data_sliced_tensor)
    acc[i][2]=calc_noisy(preds,labels)
    print("Results:",acc[i])
np.save(path+"/data/20_TRIALS_CNN_ZOOM_AUGMENTATION",acc)

Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(12000, 4, 110, 120) (12000, 1, 440, 120)
Using 4.72 GB
torch.Size([12000, 1, 440, 120])
torch.Size([12000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [240/240], Loss: 649.6819
Epoch [11/100], Step [240/240], Loss: 59.8269
Epoch [21/100], Step [240/240], Loss: 8.3617
Epoch [31/100], Step [240/240], Loss: 1.2514
Epoch [41/100], Step [240/240], Loss: 0.4467
Epoch [51/100], Step [240/240], Loss: 0.2775
Epoch [61/100], Step [240/240], Loss: 0.1973
Epoch [71/100], Step [240/240], Loss: 0.1437
Epoch [81/100], Step [240/240], Loss: 0.1166
Epoch [91/100], Step [240/240], Loss: 0.0974
Finished Training
Commence calculations...
Accuracy: 99.9989583441839 %
Accuracy: 99.99583350693722 %
Results: [0.99998958 0.99995834 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(12000, 4, 110, 120) (12000, 1, 440, 120)
Using 4.72 GB
torch.Size([12000, 1, 440, 120])
torch.Size([12000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [240/240], Loss: 649.2054
Epoch [11/100], Step [240/240], Loss: 45.1367
Epoch [21/100], Step [240/240], Loss: 5.7885
Epoch [31/100], Step [240/240], Loss: 0.9357
Epoch [41/100], Step [240/240], Loss: 0.3970
Epoch [51/100], Step [240/240], Loss: 0.4021
Epoch [61/100], Step [240/240], Loss: 0.1803
Epoch [71/100], Step [240/240], Loss: 0.1403
Epoch [81/100], Step [240/240], Loss: 0.1107
Epoch [91/100], Step [240/240], Loss: 0.0954
Finished Training
Commence calculations...
Accuracy: 99.9989583441839 %
Accuracy: 99.99583350693722 %
Results: [0.99998958 0.99995834 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(12000, 4, 110, 120) (12000, 1, 440, 120)
Using 4.72 GB
torch.Size([12000, 1, 440, 120])
torch.Size([12000, 15])


...

Epoch [1/100], Step [240/240], Loss: 649.8307
Epoch [11/100], Step [240/240], Loss: 72.6349
Epoch [21/100], Step [240/240], Loss: 9.5912
Epoch [31/100], Step [240/240], Loss: 1.8763
Epoch [41/100], Step [240/240], Loss: 0.4753
Epoch [51/100], Step [240/240], Loss: 0.2784
Epoch [61/100], Step [240/240], Loss: 0.1996
Epoch [71/100], Step [240/240], Loss: 0.1424
Epoch [81/100], Step [240/240], Loss: 0.1143
Epoch [91/100], Step [240/240], Loss: 0.0951
Finished Training
Commence calculations...
Accuracy: 99.9989583441839 %
Accuracy: 99.99583350693722 %
Results: [0.99998958 0.99995834 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(12000, 4, 110, 120) (12000, 1, 440, 120)
Using 4.72 GB
torch.Size([12000, 1, 440, 120])
torch.Size([12000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [240/240], Loss: 649.0149
Epoch [11/100], Step [240/240], Loss: 50.3929
Epoch [21/100], Step [240/240], Loss: 6.4122
Epoch [31/100], Step [240/240], Loss: 1.0203
Epoch [41/100], Step [240/240], Loss: 0.5424
Epoch [51/100], Step [240/240], Loss: 0.2784
Epoch [61/100], Step [240/240], Loss: 0.1961
Epoch [71/100], Step [240/240], Loss: 0.1393
Epoch [81/100], Step [240/240], Loss: 0.1156
Epoch [91/100], Step [240/240], Loss: 0.0930
Finished Training
Commence calculations...
Accuracy: 99.9989583441839 %
Accuracy: 99.99583350693722 %
Results: [0.99998958 0.99995834 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(12000, 4, 110, 120) (12000, 1, 440, 120)
Using 4.72 GB
torch.Size([12000, 1, 440, 120])
torch.Size([12000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [240/240], Loss: 649.6578
Epoch [11/100], Step [240/240], Loss: 61.2825
Epoch [21/100], Step [240/240], Loss: 7.5931
Epoch [31/100], Step [240/240], Loss: 1.4401
Epoch [41/100], Step [240/240], Loss: 0.6029
Epoch [51/100], Step [240/240], Loss: 0.3157
Epoch [61/100], Step [240/240], Loss: 0.2003
Epoch [71/100], Step [240/240], Loss: 0.1584
Epoch [81/100], Step [240/240], Loss: 0.1175
Epoch [91/100], Step [240/240], Loss: 0.0973
Finished Training
Commence calculations...
Accuracy: 99.9989583441839 %
Accuracy: 99.99583350693722 %
Results: [0.99998958 0.99995834 0.        ]


### experiment with different speeds

In [34]:
acc=np.zeros((num_trials,3)) #train,test,noisy

for i in range(num_trials):
    data=loaded(t=-1,filename="X_data_15.npz")
    data.applySobel()
    data.speed_augment(t=4)
    data.shuffle()
    data.X=data.X[0:15000]
    data.y=data.y[0:15000]
    train_loader,test_loader=org_data(data,(len(data.X),1,T_SIZE*data.X.shape[2],data.X.shape[3]))
    model,history=run(train_loader,0,4,num_epochs = EPOCHS)
    print("Commence calculations...")
    acc[i][0]=calc(model,train_loader)
    acc[i][1]=calc(model,test_loader)
    preds=model(test_data_sliced_tensor)
    acc[i][2]=calc_noisy(preds,labels)
    print("Results:",acc[i])
np.save(path+"/data/20_TRIALS_CNN_SPEED_AUGMENTATION",acc)

Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 484.8174
Epoch [11/100], Step [180/180], Loss: 21.6758
Epoch [21/100], Step [180/180], Loss: 0.8357
Epoch [31/100], Step [180/180], Loss: 0.3974
Epoch [41/100], Step [180/180], Loss: 0.1601
Epoch [51/100], Step [180/180], Loss: 0.1132
Epoch [61/100], Step [180/180], Loss: 0.0844
Epoch [71/100], Step [180/180], Loss: 0.0661
Epoch [81/100], Step [180/180], Loss: 0.0539
Epoch [91/100], Step [180/180], Loss: 0.0460
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 484.7044
Epoch [11/100], Step [180/180], Loss: 21.6222
Epoch [21/100], Step [180/180], Loss: 0.8060
Epoch [31/100], Step [180/180], Loss: 0.2877
Epoch [41/100], Step [180/180], Loss: 0.1665
Epoch [51/100], Step [180/180], Loss: 0.1149
Epoch [61/100], Step [180/180], Loss: 0.0842
Epoch [71/100], Step [180/180], Loss: 0.0671
Epoch [81/100], Step [180/180], Loss: 0.0544
Epoch [91/100], Step [180/180], Loss: 0.0457
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


...

Epoch [1/100], Step [180/180], Loss: 484.0479
Epoch [11/100], Step [180/180], Loss: 18.8798
Epoch [21/100], Step [180/180], Loss: 0.7507
Epoch [31/100], Step [180/180], Loss: 0.2790
Epoch [41/100], Step [180/180], Loss: 0.1623
Epoch [51/100], Step [180/180], Loss: 0.1114
Epoch [61/100], Step [180/180], Loss: 0.0834
Epoch [71/100], Step [180/180], Loss: 0.0660
Epoch [81/100], Step [180/180], Loss: 0.0534
Epoch [91/100], Step [180/180], Loss: 0.0459
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 482.5832
Epoch [11/100], Step [180/180], Loss: 14.8216
Epoch [21/100], Step [180/180], Loss: 0.7285
Epoch [31/100], Step [180/180], Loss: 0.2812
Epoch [41/100], Step [180/180], Loss: 0.1635
Epoch [51/100], Step [180/180], Loss: 0.1149
Epoch [61/100], Step [180/180], Loss: 0.0873
Epoch [71/100], Step [180/180], Loss: 0.0696
Epoch [81/100], Step [180/180], Loss: 0.0563
Epoch [91/100], Step [180/180], Loss: 0.0481
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 485.3085
Epoch [11/100], Step [180/180], Loss: 22.3345
Epoch [21/100], Step [180/180], Loss: 1.0248
Epoch [31/100], Step [180/180], Loss: 0.2992
Epoch [41/100], Step [180/180], Loss: 0.1711
Epoch [51/100], Step [180/180], Loss: 0.1142
Epoch [61/100], Step [180/180], Loss: 0.0875
Epoch [71/100], Step [180/180], Loss: 0.0684
Epoch [81/100], Step [180/180], Loss: 0.0558
Epoch [91/100], Step [180/180], Loss: 0.0473
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]


### orientation augment

In [35]:
acc=np.zeros((num_trials,3)) #train,test,noisy

for i in range(num_trials):
    data=loaded(t=T_SIZE,filename="X_data_15.npz")
    data.applySobel()
    data.orientation_augment()
    data.shuffle()
    data.X=data.X[0:15000]
    data.y=data.y[0:15000]
    train_loader,test_loader=org_data(data,(len(data.X),1,T_SIZE*data.X.shape[2],data.X.shape[3]))
    model,history=run(train_loader,0,4,num_epochs = EPOCHS)
    print("Commence calculations...")
    acc[i][0]=calc(model,train_loader)
    acc[i][1]=calc(model,test_loader)
    preds=model(test_data_sliced_tensor)
    acc[i][2]=calc_noisy(preds,labels)
    print("Results:",acc[i])
np.save(path+"/data/20_TRIALS_CNN_ORIENTATION_AUGMENTATION",acc)

Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 9000 
Window size: 4 
Image: (110, 120)
Memory needed: 0.44 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 487.0839
Epoch [11/100], Step [180/180], Loss: 91.8348
Epoch [21/100], Step [180/180], Loss: 25.3555
Epoch [31/100], Step [180/180], Loss: 9.0112
Epoch [41/100], Step [180/180], Loss: 1.1415
Epoch [51/100], Step [180/180], Loss: 0.4179
Epoch [61/100], Step [180/180], Loss: 0.1716
Epoch [71/100], Step [180/180], Loss: 0.1213
Epoch [81/100], Step [180/180], Loss: 0.0936
Epoch [91/100], Step [180/180], Loss: 0.0774
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.93889228376202 %
Results: [0.99998611 0.99938892 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 9000 
Window size: 4 
Image: (110, 120)
Memory needed: 0.44 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 486.9933
Epoch [11/100], Step [180/180], Loss: 95.8828
Epoch [21/100], Step [180/180], Loss: 24.9805
Epoch [31/100], Step [180/180], Loss: 9.0756
Epoch [41/100], Step [180/180], Loss: 1.7844
Epoch [51/100], Step [180/180], Loss: 0.5182
Epoch [61/100], Step [180/180], Loss: 0.2018
Epoch [71/100], Step [180/180], Loss: 0.1309
Epoch [81/100], Step [180/180], Loss: 0.0983
Epoch [91/100], Step [180/180], Loss: 0.0810
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 9000 
Window size: 4 
Image: (110, 120)
Memory needed: 0.44 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


...

Epoch [1/100], Step [180/180], Loss: 486.4768
Epoch [11/100], Step [180/180], Loss: 72.7296
Epoch [21/100], Step [180/180], Loss: 20.9319
Epoch [31/100], Step [180/180], Loss: 5.8562
Epoch [41/100], Step [180/180], Loss: 1.4347
Epoch [51/100], Step [180/180], Loss: 0.2821
Epoch [61/100], Step [180/180], Loss: 0.1633
Epoch [71/100], Step [180/180], Loss: 0.1198
Epoch [81/100], Step [180/180], Loss: 0.0926
Epoch [91/100], Step [180/180], Loss: 0.0743
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.93889228376202 %
Results: [0.99998611 0.99938892 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 9000 
Window size: 4 
Image: (110, 120)
Memory needed: 0.44 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 486.9614
Epoch [11/100], Step [180/180], Loss: 91.6697
Epoch [21/100], Step [180/180], Loss: 27.3929
Epoch [31/100], Step [180/180], Loss: 9.4875
Epoch [41/100], Step [180/180], Loss: 2.4540
Epoch [51/100], Step [180/180], Loss: 0.7530
Epoch [61/100], Step [180/180], Loss: 0.3061
Epoch [71/100], Step [180/180], Loss: 0.1292
Epoch [81/100], Step [180/180], Loss: 0.1001
Epoch [91/100], Step [180/180], Loss: 0.0805
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 9000 
Window size: 4 
Image: (110, 120)
Memory needed: 0.44 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 485.2367
Epoch [11/100], Step [180/180], Loss: 64.4260
Epoch [21/100], Step [180/180], Loss: 18.1979
Epoch [31/100], Step [180/180], Loss: 5.0892
Epoch [41/100], Step [180/180], Loss: 1.1270
Epoch [51/100], Step [180/180], Loss: 0.2456
Epoch [61/100], Step [180/180], Loss: 0.1621
Epoch [71/100], Step [180/180], Loss: 0.1164
Epoch [81/100], Step [180/180], Loss: 0.0902
Epoch [91/100], Step [180/180], Loss: 0.0745
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]


### no augmentation

In [36]:
acc=np.zeros((num_trials,3)) #train,test,noisy

for i in range(num_trials):
    data=loaded(t=T_SIZE,filename="X_data_15.npz")
    data.applySobel()
    data.shuffle()
    data.X=data.X[0:15000]
    data.y=data.y[0:15000]
    train_loader,test_loader=org_data(data,(len(data.X),1,T_SIZE*data.X.shape[2],data.X.shape[3]))
    model,history=run(train_loader,0,4,num_epochs = EPOCHS)
    print("Commence calculations...")
    acc[i][0]=calc(model,train_loader)
    acc[i][1]=calc(model,test_loader)
    preds=model(test_data_sliced_tensor)
    acc[i][2]=calc_noisy(preds,labels)
    print("Results:",acc[i])
np.save(path+"/data/20_TRIALS_CNN_NO_AUGMENTATION",acc)

Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(3000, 4, 110, 120) (3000, 1, 440, 120)
Using 1.18 GB
torch.Size([3000, 1, 440, 120])
torch.Size([3000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [60/60], Loss: 162.2673
Epoch [11/100], Step [60/60], Loss: 74.0245
Epoch [21/100], Step [60/60], Loss: 24.5539
Epoch [31/100], Step [60/60], Loss: 9.6093
Epoch [41/100], Step [60/60], Loss: 4.5145
Epoch [51/100], Step [60/60], Loss: 1.1680
Epoch [61/100], Step [60/60], Loss: 0.6256
Epoch [71/100], Step [60/60], Loss: 0.1830
Epoch [81/100], Step [60/60], Loss: 0.1345
Epoch [91/100], Step [60/60], Loss: 0.0963
Finished Training
Commence calculations...
Accuracy: 99.99583350693722 %
Accuracy: 99.98333611064822 %
Results: [0.99995834 0.99983336 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(3000, 4, 110, 120) (3000, 1, 440, 120)
Using 1.18 GB
torch.Size([3000, 1, 440, 120])
torch.Size([3000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [60/60], Loss: 162.3972
Epoch [11/100], Step [60/60], Loss: 77.7909
Epoch [21/100], Step [60/60], Loss: 19.4813
Epoch [31/100], Step [60/60], Loss: 5.4362
Epoch [41/100], Step [60/60], Loss: 1.7950
Epoch [51/100], Step [60/60], Loss: 0.5925
Epoch [61/100], Step [60/60], Loss: 0.2113
Epoch [71/100], Step [60/60], Loss: 0.1410
Epoch [81/100], Step [60/60], Loss: 0.1068
Epoch [91/100], Step [60/60], Loss: 0.0841
Finished Training
Commence calculations...
Accuracy: 99.99583350693722 %
Accuracy: 99.98333611064822 %
Results: [0.99995834 0.99983336 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(3000, 4, 110, 120) (3000, 1, 440, 120)
Using 1.18 GB
torch.Size([3000, 1, 440, 120])
torch.Size([3000, 15])


...

Epoch [1/100], Step [60/60], Loss: 162.0947
Epoch [11/100], Step [60/60], Loss: 66.0610
Epoch [21/100], Step [60/60], Loss: 19.0809
Epoch [31/100], Step [60/60], Loss: 7.0039
Epoch [41/100], Step [60/60], Loss: 3.3974
Epoch [51/100], Step [60/60], Loss: 0.9491
Epoch [61/100], Step [60/60], Loss: 0.2532
Epoch [71/100], Step [60/60], Loss: 0.1655
Epoch [81/100], Step [60/60], Loss: 0.1151
Epoch [91/100], Step [60/60], Loss: 0.0915
Finished Training
Commence calculations...
Accuracy: 99.99583350693722 %
Accuracy: 99.98333611064822 %
Results: [0.99995834 0.99983336 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(3000, 4, 110, 120) (3000, 1, 440, 120)
Using 1.18 GB
torch.Size([3000, 1, 440, 120])
torch.Size([3000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [60/60], Loss: 162.1230
Epoch [11/100], Step [60/60], Loss: 62.2428
Epoch [21/100], Step [60/60], Loss: 18.7486
Epoch [31/100], Step [60/60], Loss: 7.2103
Epoch [41/100], Step [60/60], Loss: 2.6880
Epoch [51/100], Step [60/60], Loss: 0.5091
Epoch [61/100], Step [60/60], Loss: 0.2447
Epoch [71/100], Step [60/60], Loss: 0.1567
Epoch [81/100], Step [60/60], Loss: 0.1115
Epoch [91/100], Step [60/60], Loss: 0.0897
Finished Training
Commence calculations...
Accuracy: 99.99583350693722 %
Accuracy: 99.98333611064822 %
Results: [0.99995834 0.99983336 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(3000, 4, 110, 120) (3000, 1, 440, 120)
Using 1.18 GB
torch.Size([3000, 1, 440, 120])
torch.Size([3000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [60/60], Loss: 162.2398
Epoch [11/100], Step [60/60], Loss: 61.3003
Epoch [21/100], Step [60/60], Loss: 15.8042
Epoch [31/100], Step [60/60], Loss: 6.3282
Epoch [41/100], Step [60/60], Loss: 2.2590
Epoch [51/100], Step [60/60], Loss: 0.5132
Epoch [61/100], Step [60/60], Loss: 0.2039
Epoch [71/100], Step [60/60], Loss: 0.1449
Epoch [81/100], Step [60/60], Loss: 0.1033
Epoch [91/100], Step [60/60], Loss: 0.0808
Finished Training
Commence calculations...
Accuracy: 99.99583350693722 %
Accuracy: 99.98333611064822 %
Results: [0.99995834 0.99983336 0.        ]


### different starts

In [37]:
acc=np.zeros((num_trials,3)) #train,test,noisy

for i in range(num_trials):
    data=loaded(t=20,filename="X_data_15.npz")
    data.applySobel()
    data.different_starts(t=4)
    data.shuffle()
    data.X=data.X[0:15000]
    data.y=data.y[0:15000]
    train_loader,test_loader=org_data(data,(len(data.X),1,T_SIZE*data.X.shape[2],data.X.shape[3]))
    model,history=run(train_loader,0,4,num_epochs = EPOCHS)
    print("Commence calculations...")
    acc[i][0]=calc(model,train_loader)
    acc[i][1]=calc(model,test_loader)
    preds=model(test_data_sliced_tensor)
    acc[i][2]=calc_noisy(preds,labels)
    print("Results:",acc[i])
np.save(path+"/data/20_TRIALS_CNN_DIFFERENT_AUGMENTATION",acc)

Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [300/300], Loss: 233.2946
Epoch [11/100], Step [300/300], Loss: 54.0207
Epoch [21/100], Step [300/300], Loss: 7.7764
Epoch [31/100], Step [300/300], Loss: 2.8667
Epoch [41/100], Step [300/300], Loss: 1.2364
Epoch [51/100], Step [300/300], Loss: 0.6719
Epoch [61/100], Step [300/300], Loss: 0.4180
Epoch [71/100], Step [300/300], Loss: 0.2379
Epoch [81/100], Step [300/300], Loss: 0.1213
Epoch [91/100], Step [300/300], Loss: 0.0976
Finished Training
Commence calculations...
Accuracy: 99.99916667361106 %
Accuracy: 99.99666677777408 %
Results: [0.99999167 0.99996667 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [300/300], Loss: 288.2931
Epoch [11/100], Step [300/300], Loss: 55.8686
Epoch [21/100], Step [300/300], Loss: 8.1964
Epoch [31/100], Step [300/300], Loss: 2.7310
Epoch [41/100], Step [300/300], Loss: 1.0827
Epoch [51/100], Step [300/300], Loss: 0.4528
Epoch [61/100], Step [300/300], Loss: 0.3275
Epoch [71/100], Step [300/300], Loss: 0.1810
Epoch [81/100], Step [300/300], Loss: 0.1389
Epoch [91/100], Step [300/300], Loss: 0.1006
Finished Training
Commence calculations...
Accuracy: 99.99916667361106 %
Accuracy: 99.99666677777408 %
Results: [0.99999167 0.99996667 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


...

Epoch [1/100], Step [300/300], Loss: 245.8284
Epoch [11/100], Step [300/300], Loss: 48.3391
Epoch [21/100], Step [300/300], Loss: 7.3886
Epoch [31/100], Step [300/300], Loss: 2.3059
Epoch [41/100], Step [300/300], Loss: 1.0353
Epoch [51/100], Step [300/300], Loss: 0.5028
Epoch [61/100], Step [300/300], Loss: 0.2175
Epoch [71/100], Step [300/300], Loss: 0.1648
Epoch [81/100], Step [300/300], Loss: 0.1266
Epoch [91/100], Step [300/300], Loss: 0.1007
Finished Training
Commence calculations...
Accuracy: 99.99916667361106 %
Accuracy: 99.99666677777408 %
Results: [0.99999167 0.99996667 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [300/300], Loss: 248.0342
Epoch [11/100], Step [300/300], Loss: 28.9127
Epoch [21/100], Step [300/300], Loss: 5.4941
Epoch [31/100], Step [300/300], Loss: 2.2648
Epoch [41/100], Step [300/300], Loss: 0.9294
Epoch [51/100], Step [300/300], Loss: 0.4592
Epoch [61/100], Step [300/300], Loss: 0.2321
Epoch [71/100], Step [300/300], Loss: 0.1534
Epoch [81/100], Step [300/300], Loss: 0.1114
Epoch [91/100], Step [300/300], Loss: 0.1023
Finished Training
Commence calculations...
Accuracy: 99.99916667361106 %
Accuracy: 99.99666677777408 %
Results: [0.99999167 0.99996667 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [300/300], Loss: 276.9045


### all augmentation

In [24]:
acc=np.zeros((num_trials,3)) #train,test,noisy

for i in range(num_trials):
    data=loaded(t=20,filename="X_data_15.npz")
    data.applySobel()
    data.speed_augment(t=4)
    data.zoom_augment([10,20,30,40])
    data.orientation_augment()
    data.shuffle()
    data.X=data.X[0:15000]
    data.y=data.y[0:15000]
    train_loader,test_loader=org_data(data,(len(data.X),1,T_SIZE*data.X.shape[2],data.X.shape[3]))
    model,history=run(train_loader,0,4,num_epochs = EPOCHS)
    print("Commence calculations...")
    acc[i][0]=calc(model,train_loader)
    acc[i][1]=calc(model,test_loader)
    preds=model(test_data_sliced_tensor)
    acc[i][2]=calc_noisy(preds,labels)
    print("Results:",acc[i])
np.save(path+"/data/20_TRIALS_CNN_ALL_AUGMENTATION",acc)

Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 108000 
Window size: 4 
Image: (110, 120)
Memory needed: 5.31 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [300/300], Loss: 812.5735
Epoch [11/100], Step [300/300], Loss: 185.3249
Epoch [21/100], Step [300/300], Loss: 41.3338
Epoch [31/100], Step [300/300], Loss: 16.5446
Epoch [41/100], Step [300/300], Loss: 6.7562
Epoch [51/100], Step [300/300], Loss: 2.3201
Epoch [61/100], Step [300/300], Loss: 1.3317
Epoch [71/100], Step [300/300], Loss: 0.6575
Epoch [81/100], Step [300/300], Loss: 0.3427
Epoch [91/100], Step [300/300], Loss: 0.1603
Finished Training
Commence calculations...
Accuracy: 99.99916667361106 %
Accuracy: 99.96333455551482 %
Results: [0.99999167 0.99963335 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 108000 
Window size: 4 
Image: (110, 120)
Memory needed: 5.31 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [300/300], Loss: 812.5056
Epoch [11/100], Step [300/300], Loss: 170.2980
Epoch [21/100], Step [300/300], Loss: 43.2937
Epoch [31/100], Step [300/300], Loss: 18.9142
Epoch [41/100], Step [300/300], Loss: 8.3716
Epoch [51/100], Step [300/300], Loss: 3.1313
Epoch [61/100], Step [300/300], Loss: 0.8895
Epoch [71/100], Step [300/300], Loss: 0.4573
Epoch [81/100], Step [300/300], Loss: 0.2650
Epoch [91/100], Step [300/300], Loss: 0.2014
Finished Training
Commence calculations...
Accuracy: 99.99916667361106 %
Accuracy: 99.99666677777408 %
Results: [0.99999167 0.99996667 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 108000 
Window size: 4 
Image: (110, 120)
Memory needed: 5.31 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


...

Epoch [1/100], Step [300/300], Loss: 812.5736
Epoch [11/100], Step [300/300], Loss: 310.8346
Epoch [21/100], Step [300/300], Loss: 73.6850
Epoch [31/100], Step [300/300], Loss: 25.3977
Epoch [41/100], Step [300/300], Loss: 11.4223
Epoch [51/100], Step [300/300], Loss: 3.8686
Epoch [61/100], Step [300/300], Loss: 0.9861
Epoch [71/100], Step [300/300], Loss: 1.0532
Epoch [81/100], Step [300/300], Loss: 0.9567
Epoch [91/100], Step [300/300], Loss: 0.1823
Finished Training
Commence calculations...
Accuracy: 99.99916667361106 %
Accuracy: 99.93000233325556 %
Results: [0.99999167 0.99930002 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 108000 
Window size: 4 
Image: (110, 120)
Memory needed: 5.31 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [300/300], Loss: 812.7279
Epoch [11/100], Step [300/300], Loss: 297.5913
Epoch [21/100], Step [300/300], Loss: 65.6399
Epoch [31/100], Step [300/300], Loss: 24.3954
Epoch [41/100], Step [300/300], Loss: 9.7347
Epoch [51/100], Step [300/300], Loss: 3.7511
Epoch [61/100], Step [300/300], Loss: 2.7195
Epoch [71/100], Step [300/300], Loss: 0.6037
Epoch [81/100], Step [300/300], Loss: 0.4336
Epoch [91/100], Step [300/300], Loss: 0.1856
Finished Training
Commence calculations...
Accuracy: 99.99916667361106 %
Accuracy: 99.93000233325556 %
Results: [0.99999167 0.99930002 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 108000 
Window size: 4 
Image: (110, 120)
Memory needed: 5.31 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [300/300], Loss: 812.4444
Epoch [11/100], Step [300/300], Loss: 206.7544
Epoch [21/100], Step [300/300], Loss: 47.6270
Epoch [31/100], Step [300/300], Loss: 19.2314
Epoch [41/100], Step [300/300], Loss: 7.3926
Epoch [51/100], Step [300/300], Loss: 3.4608
Epoch [61/100], Step [300/300], Loss: 2.6263
Epoch [71/100], Step [300/300], Loss: 0.3844
Epoch [81/100], Step [300/300], Loss: 0.2621
Epoch [91/100], Step [300/300], Loss: 0.1725
Finished Training
Commence calculations...
Accuracy: 99.99916667361106 %
Accuracy: 99.96333455551482 %
Results: [0.99999167 0.99963335 0.        ]


## viewer

In [ ]:
data=loaded(t=20,filename="X_data_15.npz")
data.applySobel()
example_vid=data.X[0]

#example_speed=

In [32]:
import pandas as pd
pd.set_option('display.width', 200)  # Adjust width
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 50)  # Adjust to your preference

orient=np.load(path+"/data/20_TRIALS_CNN_ORIENTATION_AUGMENTATION.npy")
speeds=np.load(path+"/data/20_TRIALS_CNN_SPEED_AUGMENTATION.npy")
zoom=np.load(path+"/data/20_TRIALS_CNN_ZOOM_AUGMENTATION.npy")
none=np.load(path+"/data/20_TRIALS_CNN_NO_AUGMENTATION.npy")
diff=np.load(path+"/data/20_TRIALS_CNN_DIFFERENT_AUGMENTATION.npy")
all_=np.load(path+"/data/20_TRIALS_CNN_ALL_AUGMENTATION.npy")

df = pd.DataFrame(
    np.vstack([orient, speeds, zoom, none, diff, all_]),  # Stack all datasets
    columns=["Train", "Test", "New"]
)

# Add a column indicating the dataset source
df["Dataset"] = np.repeat(["Orientation", "Speed", "Zoom","None","Varying Time","All"], 20)
# Get summary statistics for each dataset separately
summary = df.groupby("Dataset")[["Train", "Test", "New"]].agg(["mean", "std", "min", "max"])
summary=summary*100
summary=summary.round(4)
# Display summary
print(summary[['Train','Test']])

print(summary[['New']])

                Train                            Test                          
                 mean  std      min      max     mean     std      min      max
Dataset                                                                        
All           99.9992  0.0  99.9992  99.9992  99.9317  0.0513  99.7633  99.9967
None          99.9958  0.0  99.9958  99.9958  99.9833  0.0000  99.9833  99.9833
Orientation   99.9986  0.0  99.9986  99.9986  99.9778  0.0261  99.9389  99.9944
Speed         99.9986  0.0  99.9986  99.9986  99.9944  0.0000  99.9944  99.9944
Varying Time  99.9992  0.0  99.9992  99.9992  99.9967  0.0000  99.9967  99.9967
Zoom          99.9990  0.0  99.9990  99.9990  99.9958  0.0000  99.9958  99.9958
              New               
             mean  std  min  max
Dataset                         
All           0.0  0.0  0.0  0.0
None          0.0  0.0  0.0  0.0
Orientation   0.0  0.0  0.0  0.0
Speed         0.0  0.0  0.0  0.0
Varying Time  0.0  0.0  0.0  0.0
Zoom          0.

In [11]:
np.max(orient[:,2])

0.0

## stats testing

In [28]:
from scipy.stats import ranksums

datasets = {
    "orient": orient,
    "speeds": speeds,
    "zoom": zoom,
    "none": none,
    "diff" : diff,
    "all" : all_
}

# Compare datasets pairwise
pairs = [
    ("orient", "speeds"),
    ("orient", "zoom"),
    ("orient", "none"),
    ("orient", "all"),
    ("orient", "diff"),
    ("speeds", "zoom"),
    ("speeds", "none"),
    ("speeds", "diff"),
    ("speeds", "all"),
    ("zoom", "none"),
    ("zoom", "diff"),
    ("zoom", "all"),
    ("none", "diff"),
    ("none", "all"),
    ("diff", "all"),

]
alpha = 0.05

for i, col in enumerate(["Train", "Test", "New"]):
    print(f"\n=== Rank-Sum Test for {col} Accuracy ===")
    for d1, d2 in pairs:
        stat, p_value = ranksums(datasets[d1][:, i], datasets[d2][:, i])  # Compare column i
        significance = "Statistically Significant" if p_value < alpha else "Not Significant"
        print(f"{d1} vs {d2}: stat={stat:.3f}, p-value={p_value:.3f} → {significance}")


=== Rank-Sum Test for Train Accuracy ===
orient vs speeds: stat=0.000, p-value=1.000 → Not Significant
orient vs zoom: stat=-5.410, p-value=0.000 → Statistically Significant
orient vs none: stat=5.410, p-value=0.000 → Statistically Significant
orient vs all: stat=-5.410, p-value=0.000 → Statistically Significant
orient vs diff: stat=-5.410, p-value=0.000 → Statistically Significant
speeds vs zoom: stat=-5.410, p-value=0.000 → Statistically Significant
speeds vs none: stat=5.410, p-value=0.000 → Statistically Significant
speeds vs diff: stat=-5.410, p-value=0.000 → Statistically Significant
speeds vs all: stat=-5.410, p-value=0.000 → Statistically Significant
zoom vs none: stat=5.410, p-value=0.000 → Statistically Significant
zoom vs diff: stat=-5.410, p-value=0.000 → Statistically Significant
zoom vs all: stat=-5.410, p-value=0.000 → Statistically Significant
none vs diff: stat=-5.410, p-value=0.000 → Statistically Significant
none vs all: stat=-5.410, p-value=0.000 → Statistically Si